In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import datetime
from dateutil import parser, rrule
from datetime import datetime, time, date
%matplotlib notebook

# RAINDATA
using statewide data download from HCDP, its actually a total pain. The data formattng is strange, and station metadata is inconsistent. Oh well...

In [15]:

"""
Formats rainfall data and metadata from CSV files, organizing the rainfall data by station and generating a metadata summary for each station.

Args:
    raindatapath (str): Path to the directory containing the rainfall data CSV files.

Returns:
    tuple: 
        - pd.DataFrame: A DataFrame containing the formatted rainfall data, where each column corresponds to a station and rows represent dates.
        - pd.DataFrame: A DataFrame containing the metadata for each station, including station ID, coordinates, and other relevant information.
"""


def FormatHCDP_Rain_Data(raindatapath):

    Datadic = {}
    metadata_dic = {}
    
    for year in os.listdir(os.path.join("..", raindatapath)):
        for month in os.listdir(os.path.join("..", raindatapath, year)):
            file = os.listdir(os.path.join("..", raindatapath, year, month))

            # Process into a better format where columns are individual stations and y axis is date
            data = pd.read_csv(os.path.join("..", raindatapath, year, month, file[0]))

            # Extract a separaate dataframe of metadata 
            data_metaonly = data[['SKN', 'Station.Name', 'Observer', 'Network', 'Island', 'ELEV.m.',
           'LAT', 'LON', 'NCEI.id', 'NWS.id', 'NESDIS.id', 'SCAN.id', 'SMART_NODE_RF.id']]
            metadata_dic[file[0]]  = data_metaonly


            data.drop([ 'Station.Name', 'Observer', 'Network', 'Island', 'ELEV.m.',
                   'LAT', 'LON', 'NCEI.id', 'NWS.id', 'NESDIS.id', 'SCAN.id',
                   'SMART_NODE_RF.id'], axis=1, inplace=True)

            data = data.set_index("SKN", drop=True)
            data = data.transpose()

            data = data.reset_index()
            data['index'] = data['index'].astype(str)

            data['index'] = data['index'].apply(lambda x: x[1:])  # Pull out the X
            data['index'] = pd.to_datetime(data['index'], yearfirst=True)  # .dt.strftime("%Y-%m-%d")
            data.set_index("index", inplace=True)
            data.columns.name = None   # drop the wierd double axis label 

            Datadic[file[0]]  = data 

    # Produce the rainfall dataframe 
    RainData = pd.concat(Datadic.values(), sort=True)  
    # Produce the metadataframe 
    RainData_Meta = pd.concat(metadata_dic.values(), sort=True) 

    # SO herees a problem, NO field in the metadata is unique, meaning there is some f-up in the station IDentifiers.  Creating a *hoprefully* unique ID with a couple fields
    RainData_Meta['UID'] = RainData_Meta['LAT'].map('{:.6f}'.format).astype(str)+"_"+RainData_Meta['LON'].map('{:.6f}'.format).astype(str)
    RainData_Meta.drop_duplicates(subset='UID', inplace=True)     # Hoping to limit to to non-duplicate stations    

    # genearate near table outside in Arc 
    RainData_Meta["SKN"] = RainData_Meta["SKN"].astype(str)+"_R"   # First make Arc read it as a string 
    RainData_Meta.to_csv('RainData_Meta_v2.csv')
    # NOW Open in Arc and do a "Point Distance" to generate a near table for each of the reservoirs and each rain location point
    # Then need to filter pertinant fields and so a table join based on the table FIDs
    RainData_Meta.head()

    return RainData, RainData_Meta

        

In [16]:
# Run the Function
raindatapath = "Data/HCDP_Rainfall_Daily_Data/station_data"

RainData, RainData_Meta = FormatHCDP_Rain_Data(raindatapath)

In [19]:
RainData.to_csv(os.path.join("..", 'Data/HCDP_Rainfall_Daily_Data', "HCDP_RainData2018-2024.csv"))
RainData_Meta.to_csv(os.path.join("..", 'Data/HCDP_Rainfall_Daily_Data', "HCDP_Rain_METADATA_2018-2024.csv"))